# Annotation Labels 

Annotation labels specify the content of the annotations for pie and bar charts through the parameter ``labels``.

The following functions set lines, define formatting of the annotation and its size:

```python
labels=layer_labels(variables)
    .format(field, format)
    .line(template)
    .size(value)
```

## Annotation Variable List: `layer_labels(variables=['var_name_1', ..., 'var_name_N'])`

The `variables` parameter defines a list of variable names, which values will be placed line by line in the annotation. If formatting is specified for a variable from this list (with the `format()` function), it will be applied. Otherwise, the default formatting is used. Additional annotation lines can be specified using the `line()` functions.

## Formatting Annotation Fields: `layer_labels().format(field, format)`

Defines the format for displaying the value.
The format will be applied to the corresponding value specified in the line template.

### Arguments

- `field` (string): The name of the variable/aesthetics. The field name begins with `^` for aesthetics. You can specify variable names without a prefix, but the `@` prefix can be also used. For example:

  - `field = '^y'` - for y aesthetic;
  - `field = 'y'` - for variable with the name "y".

- `format` (string): The format to apply to the field. The format contains a number format (`'1.f'`) or a string template (`'{.1f}'`). The numeric format for non-numeric value will be ignored. The string template contains "replacement fields" surrounded by curly braces `{}`. Any code that is not in the braces is considered literal text, and it will be copied unchanged to the result string. If you need to include a brace character into the literal text, it can be escaped by doubling: {{ and }}. For example:

  - `.format('^color', '.1f')` -> `"17.0"`;
  - `.format('cty', '{.2f} (mpg)'))` -> `"17.00 (mpg)"`;
  - `.format('^color', '{{{.2f}}}')` -> `"{17.00}"`;
  - `.format('model', '{} {{text}}')` -> `"mustang {text}"`.

Variable's and aesthetic's formats are not interchangeable, for example, `var` format will not be applied to `aes` mapped to this variable.

## Customizing Annotation Lines: `layer_labels().line(template)`

Specifies the string template to use in an annotation.

Variables and aesthetics can be accessed via a special syntax:

- `^color` for aesthetic;
- `@year` for variable;
- `@{number of cylinders}` for a variable with spaces or non-word characters in the name;
- `@..count..` for statistics variables.

A `'^'` symbol can be escaped with a backslash; a brace character in the literal text - by doubling:

- `.line('text')` -> `"text"`;
- `.line('\^text')` -> `"^text"`;
- `.line('{{text}}')` -> `"{text}"`;
- `.line('@model')` -> `"mustang"`;
- `.line('{{@model}}')` -> `"{mustang}"`.

## Annotation Text Size: `layer_labels().size(value)`

Specifies a text size in the annotation.

In [1]:
from lets_plot import *
from lets_plot.mapping import *

In [2]:
LetsPlot.setup_html()

In [3]:
def get_data():
    import pandas as pd
    result = pd.read_csv('https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv')
    result['explode'] = [0.2 if c == 'pickup' else 0.0 for c in result['class']]
    return result

In [4]:
df = get_data()
print(df.shape)
df.head()

(234, 13)


,Unnamed: 0,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,explode
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,0.0
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,0.0
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,0.0
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,0.0
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,0.0


## Annotated Bar Chart

### Configure Layer Labels Using API [Similar to the Tooltip Configuration API](https://datalore.jetbrains.com/report/static/HZqq77cegYd.E7get_WnChZ/y8xk12VJ0sHpRHcWnrDhQO)

In [5]:
gggrid([
    ggplot(df) + \
        geom_bar(aes('trans'),
                 labels=layer_labels().line('@..count..')),
    ggplot(df) + \
        geom_bar(aes('trans', y='..n..'), stat='sum', size=0,
                 labels=layer_labels().line('@..proppct..')) + \
        coord_flip()
], widths=[1, 1.2])

### Stacked Bars

In [6]:
gggrid([
    ggplot(df) + \
        geom_bar(aes('class', fill='drv'),
                 labels=layer_labels().line('@..proppct..')
                                      .format('..proppct..', '{d} %')),
    ggplot(df) + \
        geom_bar(aes('class', fill='drv'),  position='fill',
                 labels=layer_labels().line('@..proppct..')
                                      .format('..proppct..', '{d} %')) + \
        coord_flip()
])

### Narrow Bars

Narrow bars are too thin for labels to fit in, so they are displayed externally.

Also, you can use the `label_text` parameter in `theme()` to set basic text settings for annotations, using the `element_text()` function with the following parameters:

- `family` - font family;
- `face` - font face ('plain', 'italic', 'bold', 'bold_italic');
- `size` - text size, can also be specified via `layer_labels().size()`;
- `color` - text color.

In [7]:
label_text_options = element_text(family='Rockwell', face='bold', size=16, color='#542788')

In [8]:
ggplot(df, aes('class', fill='drv')) + \
    geom_bar(labels=layer_labels().line('@..proppct..').format('..proppct..', '{d} %'),
             position='dodge') + \
    theme(label_text=label_text_options) + \
    ggsize(700, 400)

## Annotated Pie Chart

In [9]:
ggplot(df) + \
    geom_pie(aes(fill='class'), size=22, hole=.2,
             labels=layer_labels().line('@class'),
             tooltips='none') + \
    scale_fill_brewer(palette='Dark2') + \
    theme_void() + \
    theme(legend_position='none')

Let's add multiline annotations.

In [10]:
ggplot(df) + \
    geom_pie(aes(fill='class'), size=22, hole=.2,
             labels=layer_labels().line('@class')
                                  .line('(@{..prop..})')
                                  .format('..prop..', '.0%'),
             tooltips='none') + \
    scale_fill_brewer(palette='Dark2') + \
    theme_void() + \
    theme(legend_position='none')

You can change labels size with `size()` function.

In [11]:
ggplot(df) + \
    geom_pie(aes(fill=as_discrete('class', order_by='..count..')), 
             size=20, hole=.2,
             labels=layer_labels().line('@..proppct..')
                                  .size(18),
             tooltips='none') + \
    scale_fill_brewer(palette='Dark2') + \
    theme_void()

In [12]:
ggplot(df) + \
    geom_pie(aes(fill=as_discrete('class', order_by='..count..', order=1)),
             size=20, hole=.3,
             labels=layer_labels().line('@..proppct..'),
             tooltips='none') + \
    theme_void() + \
    theme(label_text=label_text_options, legend_position='right') 

### Exploded Pie Chart

In [13]:
ggplot(df) + \
    geom_pie(aes(fill='class', explode='explode'),
             color='black', size=20, hole=.3,
             labels=layer_labels(['class', '..count..']),
             tooltips='none') + \
    scale_fill_gradient(low='dark_blue', high='light_green') + \
    theme_void() + theme(legend_position='none')

### Annotations on Scatter Pie Plot

In [14]:
ggplot(df, aes('drv', as_discrete('year'))) + \
    geom_pie(aes(fill='class'),
             labels=layer_labels(['..proppct..']),
             tooltips='none') + \
    ggsize(1000, 400)

### Pie Chart with Variable Sector Sizes

In [15]:
ggplot(df) + \
    geom_pie(aes(fill='class', size='..count..'),
             labels=layer_labels(['class', '..count..']),
             tooltips='none') + \
    scale_fill_brewer(palette='Dark2') + \
    scale_size(range=[15, 25]) + \
    theme_void() + \
    theme(legend_position='none')